In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import statsmodels.formula.api as sm
!pip install dmba
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

import matplotlib.pylab as plt
%matplotlib inline

no display found. Using non-interactive Agg backend


In [2]:
boston_df = pd.read_csv('BostonHousing.csv')

In [3]:
print('Number of rows and columns in data set:', 
    boston_df.shape )

Number of rows and columns in data set: (506, 14)


In [4]:
print('Original column titles:')
boston_df.columns

Original column titles:


Index(['CRIME', 'ZONE', 'INDUST', 'CHAR RIV', 'NIT OXIDE', 'ROOMS', 'AGE',
       'DISTANCE', 'RADIAL', 'TAX', 'ST RATIO', 'LOW STAT', 'MVALUE',
       'C MVALUE'],
      dtype='object')

In [5]:
print('Modified column titles with no space and one word for titles:')
boston_df.columns = [s.strip().replace(' ', '_') for s in boston_df.columns]
boston_df.columns

Modified column titles with no space and one word for titles:


Index(['CRIME', 'ZONE', 'INDUST', 'CHAR_RIV', 'NIT_OXIDE', 'ROOMS', 'AGE',
       'DISTANCE', 'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'MVALUE',
       'C_MVALUE'],
      dtype='object')

In [6]:
boston_df.dtypes

CRIME        float64
ZONE         float64
INDUST       float64
CHAR_RIV      object
NIT_OXIDE    float64
ROOMS        float64
AGE          float64
DISTANCE     float64
RADIAL         int64
TAX            int64
ST_RATIO     float64
LOW_STAT     float64
MVALUE       float64
C_MVALUE      object
dtype: object

In [7]:
# The REMODEL column is 'object'; does not have 
# the 'category' definition.
print('Original CHAR_RIV variable:')
print(boston_df.CHAR_RIV.dtype)

# Need to change REMODEL variable type to 'category'. 
boston_df.CHAR_RIV = boston_df.CHAR_RIV.astype('category')

# Display category levels (classes) and category type.
print(' ')
print('Category levels and changed variable type:')
print(boston_df.CHAR_RIV.cat.categories)  # It can take one of three levels.
print(boston_df.CHAR_RIV.dtype)  # Type is now 'category'.

Original CHAR_RIV variable:
object
 
Category levels and changed variable type:
Index(['N', 'Y'], dtype='object')
category


In [8]:
# The REMODEL column is 'object'; does not have 
# the 'category' definition.
print('Original C_MVALUE variable:')
print(boston_df.C_MVALUE.dtype)

# Need to change REMODEL variable type to 'category'. 
boston_df.C_MVALUE = boston_df.C_MVALUE.astype('category')

# Display category levels (classes) and category type.
print(' ')
print('Category levels and changed variable type:')
print(boston_df.C_MVALUE.cat.categories)  # It can take one of three levels.
print(boston_df.C_MVALUE.dtype)  # Type is now 'category'.

Original C_MVALUE variable:
object
 
Category levels and changed variable type:
Index(['No', 'Yes'], dtype='object')
category


In [9]:
boston_df_reg = pd.get_dummies(boston_df, prefix_sep='_', 
                            drop_first=True)
boston_df_reg.columns

Index(['CRIME', 'ZONE', 'INDUST', 'NIT_OXIDE', 'ROOMS', 'AGE', 'DISTANCE',
       'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'MVALUE', 'CHAR_RIV_Y',
       'C_MVALUE_Yes'],
      dtype='object')

In [10]:
np.round(boston_df_reg.describe(), decimals=2)

,CRIME,ZONE,INDUST,NIT_OXIDE,ROOMS,AGE,DISTANCE,RADIAL,TAX,ST_RATIO,LOW_STAT,MVALUE,CHAR_RIV_Y,C_MVALUE_Yes
count,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00
mean,3.61,11.36,11.14,0.55,6.28,68.57,3.80,9.55,408.24,18.46,12.65,22.53,0.07,0.17
std,8.60,23.32,6.86,0.12,0.70,28.15,2.11,8.71,168.54,2.16,7.14,9.20,0.25,0.37
min,0.01,0.00,0.46,0.38,3.56,2.90,1.13,1.00,187.00,12.60,1.73,5.00,0.00,0.00
25%,0.08,0.00,5.19,0.45,5.89,45.02,2.10,4.00,279.00,17.40,6.95,17.02,0.00,0.00
50%,0.26,0.00,9.69,0.54,6.21,77.50,3.21,5.00,330.00,19.05,11.36,21.20,0.00,0.00
75%,3.68,12.50,18.10,0.62,6.62,94.07,5.19,24.00,666.00,20.20,16.96,25.00,0.00,0.00
max,88.98,100.00,27.74,0.87,8.78,100.00,12.13,24.00,711.00,22.00,37.97,50.00,1.00,1.00


In [11]:
# Check for missing records (values) in the columns
missing_records = boston_df.isnull().sum()
print("\nMissing records in columns:")
print(missing_records)


Missing records in columns:
CRIME        0
ZONE         0
INDUST       0
CHAR_RIV     0
NIT_OXIDE    0
ROOMS        0
AGE          0
DISTANCE     0
RADIAL       0
TAX          0
ST_RATIO     0
LOW_STAT     0
MVALUE       0
C_MVALUE     0
dtype: int64


In [16]:
# Identify predictors and outcome of the regression model.
predictors = ['CRIME', 'ZONE', 'INDUST', 'NIT_OXIDE', 'ROOMS', 'AGE', 'DISTANCE',
       'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'CHAR_RIV_Y',
       'C_MVALUE_Yes']
outcome = 'MVALUE'

# Identify X and y variables for regression and partition data
# using 70% of records for training and 30% for validation 
# (test_size=0.3). 
X = boston_df_reg[predictors]
y = boston_df_reg[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.3, random_state=1)

# Create multiple linear regression model using X and y
# and LinearRegression() function from sklearn (skikit-learn) 
# library.
boston_lm = LinearRegression()
boston_lm.fit(train_X, train_y)

# Display intercept and regression coefficients. Round
# them to 2 decimals.
print('Regression Model for BostonHousing Training Set')
print()
print('Intercept: ', np.round(boston_lm.intercept_, 2)) # coefficient Bo
print(pd.DataFrame({'Predictor': X.columns, 'Coefficient': np.round(boston_lm.coef_, 2)}))

Regression Model for BostonHousing Training Set

Intercept:  48.62
       Predictor  Coefficient
0          CRIME        -0.15
1           ZONE        -0.01
2         INDUST         0.13
3      NIT_OXIDE       -17.86
4          ROOMS         0.33
5            AGE        -0.01
6       DISTANCE        -0.66
7         RADIAL         0.22
8            TAX        -0.01
9       ST_RATIO        -0.63
10      LOW_STAT        -0.47
11    CHAR_RIV_Y         2.33
12  C_MVALUE_Yes        12.13


In [17]:
# Use predict() to score (make) predictions for validation set.
boston_lm_pred = boston_lm.predict(valid_X)

# Develop and display data frame with actual values of Price,
# scoring (predicted) results, and residuals.
# Use round() function to round vlaues in data frame to 
# 2 decimals. 
print('Actual, Prediction, and Residual Prices for Validation Set')
result = round(pd.DataFrame({'Actual': valid_y,'Predicted': boston_lm_pred, 
                       'Residual': valid_y - boston_lm_pred}), 2)
print(result.head(10))

Actual, Prediction, and Residual Prices for Validation Set
     Actual  Predicted  Residual
307    28.2      24.63      3.57
343    23.9      22.03      1.87
47     16.6      17.97     -1.37
67     22.0      22.20     -0.20
362    20.8      19.42      1.38
132    23.0      19.32      3.68
292    27.9      24.68      3.22
31     14.5      17.73     -3.23
218    21.5      22.69     -1.19
90     22.6      23.09     -0.49


In [18]:
# Use predict() function to make predictions for
# training set.
pred_y = boston_lm.predict(train_X)

# Create prediction performance measures for training set.
r2 = round(r2_score(train_y, pred_y),3)
adj_r2 = round(adjusted_r2_score(train_y, pred_y, boston_lm),3)
aic = round(AIC_score(train_y, pred_y, boston_lm),2)
bic = round(BIC_score(train_y, pred_y, boston_lm),2)

# Display prediction performance measures for training set.
print('Prediction Performance Measures for Training Set')
print('r2 : ', r2)
print('Adjusted r2 : ', adj_r2)
print('AIC : ', aic)
print('BIC : ', bic)
print() 

# Create prediction performance measures for validation set.
r2 = round(r2_score(valid_y, boston_lm_pred),3)
adj_r2 = round(adjusted_r2_score(valid_y, boston_lm_pred, boston_lm),3)
aic = round(AIC_score(valid_y, boston_lm_pred, boston_lm),2)
bic = round(BIC_score(valid_y, boston_lm_pred, boston_lm),2)

# Display prediction performance measures for validation set.
print('Prediction Performance Measures for Validation Set')
print('r2 : ', r2)
print('adjusted r2 : ', adj_r2)
print('AIC : ', aic)
print('BIC : ', bic)

Prediction Performance Measures for Training Set
r2 :  0.83
Adjusted r2 :  0.824
AIC :  1963.68
BIC :  2021.72

Prediction Performance Measures for Validation Set
r2 :  0.852
adjusted r2 :  0.838
AIC :  858.0
BIC :  903.36


In [19]:
# Display common accuracy measures for training set.
print('Accuracy Measures for Training Set - All Variables')
regressionSummary(train_y, pred_y)
print()

# Display common accuracy measures for validation set.
print('Accuracy Measures for Validation Set - All Variables')
regressionSummary(valid_y, boston_lm_pred)

Accuracy Measures for Training Set - All Variables

Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 3.7145
            Mean Absolute Error (MAE) : 2.6931
          Mean Percentage Error (MPE) : -2.7567
Mean Absolute Percentage Error (MAPE) : 13.2197

Accuracy Measures for Validation Set - All Variables

Regression statistics

                      Mean Error (ME) : 0.3667
       Root Mean Squared Error (RMSE) : 3.6868
            Mean Absolute Error (MAE) : 2.7428
          Mean Percentage Error (MPE) : -2.9628
Mean Absolute Percentage Error (MAPE) : 13.9356


In [21]:
# Define train_model() function used in Executive Search
# algorithm with executive_search() function. 
def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

# Define score_model() function used in Executive Search
# algorithm with executive_search() function. 
def score_model(model, variables):
    pred_y = model.predict(train_X[variables])
    # Apply negative sign as score is optimized to be 
    # as low as possible in exhaustive_search() function.
    return -adjusted_r2_score(train_y, pred_y, model)

# Create allVariables object with predcitors in train_X,
allVariables = train_X.columns

# The exhaustive_search() function :
results = exhaustive_search(allVariables, train_model, score_model)

# Create data[] loop process to identify and append the best model 
data = []
for result in results:
    model = result['model']
    variables = result['variables']
    AIC = AIC_score(train_y, model.predict(train_X[variables]), model)
    d = {'n': result['n'], 'r2adj': -result['score'], 'AIC': AIC}
    d.update({var: var in result['variables'] for var in allVariables})
    data.append(d)

# Define the width of output presentation to be wider
pd.set_option('display.width', 100)

# Display the Exhaustive Search results.
print(pd.DataFrame(data, columns=('n', 'r2adj', 'AIC') + tuple(sorted(allVariables))))

# Reset the output width to the default. 
pd.reset_option('display.width')

     n     r2adj          AIC    AGE  CHAR_RIV_Y  CRIME  C_MVALUE_Yes  DISTANCE  INDUST  LOW_STAT  \
0    1  0.615757  2227.470343  False       False  False          True     False   False     False   
1    2  0.784502  2023.736517  False       False  False          True     False   False      True   
2    3  0.793737  2009.222342  False       False   True          True     False   False      True   
3    4  0.800829  1997.822810  False        True   True          True     False   False      True   
4    5  0.804618  1992.008003  False       False  False          True      True   False      True   
5    6  0.811403  1980.477479  False        True  False          True      True   False      True   
6    7  0.816868  1971.047129  False        True   True          True      True   False      True   
7    8  0.822139  1961.682655  False        True   True          True      True   False      True   
8    9  0.822845  1961.248007  False        True   True          True      True    True    

In [22]:
# Develop the multiple linear regression model based
# on the Exhaustive Search results.

# Identify predictors and outcome of the regression model.
predictors_ex = ['CRIME', 'ZONE', 'INDUST', 'NIT_OXIDE',  'DISTANCE',
       'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'CHAR_RIV_Y',
       'C_MVALUE_Yes']
outcome = 'MVALUE'

# Identify X and y variables for regression and partition data
X = boston_df_reg[predictors_ex]
y = boston_df_reg[outcome]
train_X_ex, valid_X_ex, train_y_ex, valid_y_ex = \
          train_test_split(X, y, test_size=0.3, random_state=1)

# Create multiple linear regression model using X and y.
boston_ex = LinearRegression()
boston_ex.fit(train_X_ex, train_y_ex)

# Display intercept and regression coefficients. Round them
# to 2 decimals.
print('Regression Model for Training Set Using Exhaustive Search')
print()
print('Intercept ', np.round(boston_ex.intercept_, 2))
print(pd.DataFrame({'Predictor': X.columns,
            'Coefficient': np.round(boston_ex.coef_, 2)}))

Regression Model for Training Set Using Exhaustive Search

Intercept  50.9
       Predictor  Coefficient
0          CRIME        -0.15
1           ZONE        -0.01
2         INDUST         0.13
3      NIT_OXIDE       -18.45
4       DISTANCE        -0.64
5         RADIAL         0.22
6            TAX        -0.01
7       ST_RATIO        -0.65
8       LOW_STAT        -0.49
9     CHAR_RIV_Y         2.35
10  C_MVALUE_Yes        12.30


In [24]:
# Use predict() function to score (make) predictions 
# for validation set and measure their accuracy using
# Exhaustive Search algorithm.
boston_ex_pred = boston_ex.predict(valid_X_ex)

# Develop and display data frame with actual values of Price,
# scoring (predicted) results, and residuals.
# Use round() function to round vlaues in data frame to 
# 2 decimals. 
result = round(pd.DataFrame({'Actual': valid_y_ex,'Predicted': boston_ex_pred, 
                       'Residual': valid_y_ex - boston_ex_pred}), 2)
print()
print('Prediction for Validation Set Using Exhaustive Search') 
print(result.head(10))

# Display common accuracy measures for validation set.
print()
print('Accuracy Measures for Validation Set Using Exhaustive Search')
regressionSummary(valid_y_ex, boston_ex_pred)


Prediction for Validation Set Using Exhaustive Search
     Actual  Predicted  Residual
307    28.2      24.59      3.61
343    23.9      22.02      1.88
47     16.6      18.09     -1.49
67     22.0      22.18     -0.18
362    20.8      19.82      0.98
132    23.0      19.32      3.68
292    27.9      24.54      3.36
31     14.5      17.92     -3.42
218    21.5      22.82     -1.32
90     22.6      23.06     -0.46

Accuracy Measures for Validation Set Using Exhaustive Search

Regression statistics

                      Mean Error (ME) : 0.3931
       Root Mean Squared Error (RMSE) : 3.7376
            Mean Absolute Error (MAE) : 2.7717
          Mean Percentage Error (MPE) : -2.8476
Mean Absolute Percentage Error (MAPE) : 14.0064


In [25]:
# Define train_model() function used in Backward Elimination
# algorithm with backward_elimination() function. 
def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

# Define score_model() function used in Backward Elimination
# algorithm with backward_elimination() function. 
def score_model(model, variables):
    return AIC_score(train_y, model.predict(train_X[variables]), model)

# Use backward_elimination() function to identify the
# best_model and best_variables. 
best_model_be, best_variables_be = backward_elimination(train_X.columns, 
                        train_model, score_model, verbose=True)

# Display best variables based on Backward Elimination algorithm. 
print()
print('Best Variables from Backward Elimination Algorithm')
print(best_variables_be)

Variables: CRIME, ZONE, INDUST, NIT_OXIDE, ROOMS, AGE, DISTANCE, RADIAL, TAX, ST_RATIO, LOW_STAT, CHAR_RIV_Y, C_MVALUE_Yes
Start: score=1963.68
Step: score=1962.03, remove AGE
Step: score=1960.30, remove ZONE
Step: score=1958.80, remove ROOMS
Step: score=1958.80, remove None

Best Variables from Backward Elimination Algorithm
['CRIME', 'INDUST', 'NIT_OXIDE', 'DISTANCE', 'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'CHAR_RIV_Y', 'C_MVALUE_Yes']


In [27]:
# Develop the multiple linear regression model based
# on the Backward Elimination results.

# Identify predictors and outcome of the regression model.
predictors_be = ['CRIME', 'INDUST', 'NIT_OXIDE', 'DISTANCE', 
                 'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'CHAR_RIV_Y', 'C_MVALUE_Yes']
outcome = 'MVALUE'

# Identify X and y variables for regression and partition data
# using 60% of records for training and 40% for validation 
# (test_size=0.4). 
X = boston_df_reg[predictors_be]
y = boston_df_reg[outcome]
train_X_be, valid_X_be, train_y_be, valid_y_be = \
          train_test_split(X, y, test_size=0.3, random_state=1)

# Create multiple linear regression model using X and y.
boston_be = LinearRegression()
boston_be.fit(train_X_be, train_y_be)

# Display intercept and regression coefficients. Round them
# to 2 decimals.
print('Regression Model for Training Set Using Backward Elimination')
print()
print('Intercept ', np.round(boston_be.intercept_, 2))
print(pd.DataFrame({'Predictor': X.columns,
            'Coefficient': np.round(boston_be.coef_, 2)}))

Regression Model for Training Set Using Backward Elimination

Intercept  50.82
      Predictor  Coefficient
0         CRIME        -0.15
1        INDUST         0.13
2     NIT_OXIDE       -18.39
3      DISTANCE        -0.69
4        RADIAL         0.23
5           TAX        -0.01
6      ST_RATIO        -0.63
7      LOW_STAT        -0.49
8    CHAR_RIV_Y         2.34
9  C_MVALUE_Yes        12.19


In [28]:
# Use predict() to score predictions for validation set in
# regression model based Backward Elimination algorithm.
boston_be_pred = boston_be.predict(valid_X_be)

# Develop and display data frame with actual values of Price,
# scoring (predicted) results, and residuals.
# Use round() function to round vlaues in data frame to 
# 2 decimals. 
result = round(pd.DataFrame({'Actual': valid_y_be,'Predicted': boston_be_pred, 
                       'Residual': valid_y_be - boston_be_pred}), 2)
print()
print('Predictions for Validation Set Using Backward Elimination')
print(result.head(10))

# Display common accuracy measures for validation set.
print()
print('Accuracy Measures for Validation Set Using Backward Elimination')
regressionSummary(valid_y_be, boston_be_pred)


Predictions for Validation Set Using Backward Elimination
     Actual  Predicted  Residual
307    28.2      24.89      3.31
343    23.9      22.22      1.68
47     16.6      17.95     -1.35
67     22.0      22.06     -0.06
362    20.8      19.87      0.93
132    23.0      19.39      3.61
292    27.9      25.07      2.83
31     14.5      17.91     -3.41
218    21.5      22.81     -1.31
90     22.6      23.04     -0.44

Accuracy Measures for Validation Set Using Backward Elimination

Regression statistics

                      Mean Error (ME) : 0.3854
       Root Mean Squared Error (RMSE) : 3.7318
            Mean Absolute Error (MAE) : 2.7591
          Mean Percentage Error (MPE) : -2.8698
Mean Absolute Percentage Error (MAPE) : 13.9371


In [29]:
# Display common accuracy measures for training set.
print('Accuracy Measures for Training Set - All Variables')
regressionSummary(train_y, pred_y)
print()

# Display common accuracy measures for validation set.
print('Accuracy Measures for Validation Set - All Variables')
regressionSummary(valid_y, boston_lm_pred)

# Display common accuracy measures for validation set.
print()
print('Accuracy Measures for Validation Set Using Exhaustive Search')
regressionSummary(valid_y_ex, boston_ex_pred)

print()
print('Accuracy Measures for Validation Set Using Backward Elimination')
regressionSummary(valid_y_be, boston_be_pred)

Accuracy Measures for Training Set - All Variables

Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 3.7145
            Mean Absolute Error (MAE) : 2.6931
          Mean Percentage Error (MPE) : -2.7567
Mean Absolute Percentage Error (MAPE) : 13.2197

Accuracy Measures for Validation Set - All Variables

Regression statistics

                      Mean Error (ME) : 0.3667
       Root Mean Squared Error (RMSE) : 3.6868
            Mean Absolute Error (MAE) : 2.7428
          Mean Percentage Error (MPE) : -2.9628
Mean Absolute Percentage Error (MAPE) : 13.9356

Accuracy Measures for Validation Set Using Exhaustive Search

Regression statistics

                      Mean Error (ME) : 0.3931
       Root Mean Squared Error (RMSE) : 3.7376
            Mean Absolute Error (MAE) : 2.7717
          Mean Percentage Error (MPE) : -2.8476
Mean Absolute Percentage Error (MAPE) : 14.0064

Accuracy Measures for Validation Set Using Backwa